# Search the entire ALMA archive for some target lines using the `ALMAxmatch` module's `runQueryWithLines` method

### Import `astroquery.alma` and `ALMAxmatch` modules
- This requires adding to the python path for now

### Set path to custom `astroquery` module
This is a temporary hack to "use a version of ```astroquery``` that does not have a bug in the release date column. 
To get this set up run the following in terminal:

`git clone https://github.com/astropy/astroquery.git`

`cd astroquery`

`python setup.py build`

`python setup.py install`

replace `python` with `python3` if necessary

In [1]:
import sys
sys.path = ['/Users/thbrown/astroquery'] + sys.path

Import module and print valid payload keywords

In [2]:
from astroquery.alma import Alma
Alma.help()

Valid ALMA keywords.  Left column is the description, right column is the name of the keyword to pass to astroquery.alma queries:

Position
  Source name (Resolver)           : source_name_resolver               
  Source name (ALMA)               : source_name_alma                   
  RA Dec                           : ra_dec                             
  Galactic                         : galactic                           
  Angular resolution               : spatial_resolution                 
  Largest angular scale            : spatial_scale_max                  
  Field of view                    : fov                                

Energy
  Frequency                        : frequency                          
  Bandwidth                        : bandwidth                          
  Spectral resolution              : spectral_resolution                
  Band                             : band_list            -> 3(84-116 GHz)=3, 4(125-163 GHz)=4, 5(163-211 GHz)=5, 6(211-27

In [3]:
import sys
sys.path = ['/Users/thbrown/ALMA/think_tank_code/tools/archiveDev'] + sys.path
from ALMAxmatch import archiveSearch

## Run the archive search
1. Define the line rest frequencies (names are optional, output defaults to *line 0*, *line 1*, ...) and redshift range of search
2. Calculate min and max frequencies for lines in the redshift range
3. Create a dictionary of payload keywords (including frequency) that are accepted by the ALMA archive system.
2. Query the service using `alma.query` and return a table object, selecting only science data

### 2. Define lines of interest and redshift

Search for the following lines at $ z\leq 0.1$:

1. $^{12}$CO ($J=1-0$) @ 115.27120180 GHz rest frequency
2. $^{13}$CO ($J=1-0$) @ 109.78217340 GHz rest frequency
3. C$^{18}$O ($J=1-0$) @ 110.20135430 GHz rest frequency

In [4]:
# rest frame frequencies
rf_12CO_10 = 115.27120180 # 12CO J=1-0
rf_13CO_10 = 110.20135430 # 13CO J=1-0
rf_C18O_10 = 109.78217340 # C18O J=1-0

rest_frequencies = [rf_12CO_10, rf_13CO_10, rf_C18O_10]
line_names = ['12CO J=1-0', '13CO J=1-0', 'C18O J=1-0'] # column names for observed boolean flags 

In [5]:
redshift_range = [0.05,0.055]

In [6]:
import numpy as np
def _observedFreq(restFreq, z):
        """Return observed frequency according to nu_0 / nu = 1 + z."""
        return restFreq/(1+z)
    
# sort redshift range and frequency arrays
restFreqs = np.array(rest_frequencies)
restFreqs.sort()

redshiftRange = np.array(redshift_range)
redshiftRange.sort()
        
# define frequency range from lines and redshifts
lowFreq = _observedFreq(restFreqs[0], redshiftRange[1])
highFreq = _observedFreq(restFreqs[-1], redshiftRange[0])
freqLimits = '{:} .. {:}'.format(lowFreq, highFreq)

print('the observered frequency range of interest is '+freqLimits+' GHz')

the observered frequency range of interest is 104.05893213270143 .. 109.78209695238095 GHz


In [7]:
payload = {'frequency':freqLimits, }

testQuery = Alma.query(payload=payload, science=True)

In [8]:
testQuery

Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64
2011.0.00957.S,SPT_0103-45,15.797875000000001,-45.64827777777778,296.5034970667741,-71.3330336577803,[3],8.004940808270794,976.657768509833,12m,,151.2,2013-04-25,"[91.67..93.55GHz,976.66kHz,7.4mJy/beam@10km/s,300.4uJy/beam@native, XX YY] U [93.55..95.42GHz,976.66kHz,7.3mJy/beam@10km/s,301.1uJy/beam@native, XX YY] U [103.67..105.55GHz,976.66kHz,7.3mJy/beam@10km/s,313.2uJy/beam@native, XX YY] U [105.55..107.42GHz,976.66kHz,7.3mJy/beam@10km/s,318.9uJy/beam@native, XX YY]",2725.895948314194,XX YY,2011-11-19 03:43:05,"Weiss, Axel",Tuning3_1-2HA,"Chapman, Scott; Brodwin, Mark; Greve, Thomas; Hezaveh, Yashar; Sharon, Keren; Ashby, Matthew; Menten, Karl; Carlstrom, John; Marrone, Dan; De Breuck, Carlos; Rosenman, Michael; Holzapfel, William; Vieira, Joaquin; Crawford, Thomas; Lupu, Roxana; Fassnacht, Chris; Holder, Gilbert; Aguirre, James;",7.260172793218402,0.1540486568273702,4.0281453,,uid://A001/X74/X291,uid://A002/X321719/X723,The ALMA-SPT Redshift Survey,S,TARGET WVR,63.28655495409857,32.80138244373298,Y,14,"Sub-mm Galaxies (SMG), Gravitational lenses",Galaxy evolution,2011.0.00957.S
2011.0.00957.S,SPT_0103-45,15.797875000000001,-45.64827777777778,296.5034970667741,-71.3330336577803,[3],10.727745046324552,976.6579716976725,12m,,151.2,2013-04-25,"[95.42..97.30GHz,976.66kHz,7.3mJy/beam@10km/s,304.2uJy/beam@native, XX YY] U [97.30..99.17GHz,976.66kHz,7.2mJy/beam@10km/s,303uJy/beam@native, XX YY] U [107.42..109.30GHz,976.66kHz,7.5mJy/beam@10km/s,327.9uJy/beam@native, XX YY] U [109.30..111.17GHz,976.66kHz,7.8mJy/beam@10km/s,343.8uJy/beam@native, XX YY]",2633.9398359361962,XX YY,2011-11-19 04:46:11,"Weiss, Axel",Tuning4_1-2HA,"Chapman, Scott; Brodwin, Mark; Greve, Thomas; Hezaveh, Yashar; Sharon, Keren; Ashby, Matthew; Menten, Karl; Carlstrom, John; Marrone, Dan; De Breuck, Carlos; Rosenman, Michael; Holzapfel, William; Vieira, Joaquin; Crawford, Thomas; Lupu, Roxana; Fassnacht, Chris; Holder, Gilbert; Aguirre, James;",7.468943547865293,0.15921203104645965,4.161038,,uid://A001/X74/X293,uid://A002/X321719/X826,The ALMA-SPT Redshift Survey,S,TARGET WVR,60.988833626470466,43.958459712754184,Y,14,"Sub-mm Galaxies (SMG), Gravitational lenses",Galaxy evolution,2011.0.00957.S
2011.0.00957.S,SPT_0113-46,18.285375,-46.2963611111111,291.725192459293,-70.35680731865747,[3],7.806784683409882,976.657768509833,12m,,151.2,2013-04-25,"[91.67..93.55GHz,976.66kHz,7.4mJy/beam@10km/s,300.6uJy/beam@native, XX YY] U [93.55..95.42GHz,976.66kHz,7.3mJy/beam@10km/s,301.3uJy/beam@native, XX YY] U [103.67..105.55GHz,976.66kHz,7.3mJy/beam@10km/s,313.5uJy/beam@native, XX YY] U [105.55..107.42GHz,976.66kHz,7.3mJy/beam@10km/s,319.2uJy/beam@native, XX YY]",2725.895948314194,XX YY,2011-11-19 03:43:05,"Weiss, Axel",Tuning3_1-2HA,"Chapman, Scott; Brodwin, Mark; Greve, Thomas; Hezaveh, Yashar; Sharon, Keren; Ashby, Matthew; Menten, Karl; Carlstrom, John; Marrone, Dan; De Breuck, Carlos; Rosenman, Michael; Holzapfel, William; Vieira, Joaquin; Crawford, Thomas; Lupu, Roxana; Fassnacht, Chris; Holder, Gilbert; Aguirre, James;",7.266922806270144,0.15418158889140254,4.0281453,,uid://A001/X74/X291,uid://A002/X321719/X723,The ALMA-SPT Redsh

In [9]:
# sanitize ALMA source names
safeNames = testQuery['Source name']
safeNames = np.char.replace(safeNames, b' ', b'')
safeNames = np.char.replace(safeNames, b'_', b'')
safeNames = np.char.upper(safeNames)
testQuery['ALMA sanitized source name'] = safeNames

In [10]:
testQuery

Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE,ALMA sanitized source name
,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcs,arcs,,,,,,
bytes64,bytes256,float64,float64,float64,float64,object,float64,float64,object,object,float64,object,bytes2700,float64,bytes64,object,bytes256,bytes128,bytes512,float64,float64,float32,bytes64,bytes64,bytes32,bytes256,bytes16,bytes256,float64,float64,bytes1,int64,bytes200,bytes200,bytes64,bytes29
2011.0.00957.S,SPT_0103-45,15.797875000000001,-45.64827777777778,296.5034970667741,-71.3330336577803,[3],8.004940808270794,976.657768509833,12m,,151.2,2013-04-25,"[91.67..93.55GHz,976.66kHz,7.4mJy/beam@10km/s,300.4uJy/beam@native, XX YY] U [93.55..95.42GHz,976.66kHz,7.3mJy/beam@10km/s,301.1uJy/beam@native, XX YY] U [103.67..105.55GHz,976.66kHz,7.3mJy/beam@10km/s,313.2uJy/beam@native, XX YY] U [105.55..107.42GHz,976.66kHz,7.3mJy/beam@10km/s,318.9uJy/beam@native, XX YY]",2725.895948314194,XX YY,2011-11-19 03:43:05,"Weiss, Axel",Tuning3_1-2HA,"Chapman, Scott; Brodwin, Mark; Greve, Thomas; Hezaveh, Yashar; Sharon, Keren; Ashby, Matthew; Menten, Karl; Carlstrom, John; Marrone, Dan; De Breuck, Carlos; Rosenman, Michael; Holzapfel, William; Vieira, Joaquin; Crawford, Thomas; Lupu, Roxana; Fassnacht, Chris; Holder, Gilbert; Aguirre, James;",7.260172793218402,0.1540486568273702,4.0281453,,uid://A001/X74/X291,uid://A002/X321719/X723,The ALMA-SPT Redshift Survey,S,TARGET WVR,63.28655495409857,32.80138244373298,Y,14,"Sub-mm Galaxies (SMG), Gravitational lenses",Galaxy evolution,2011.0.00957.S,SPT0103-45
2011.0.00957.S,SPT_0103-45,15.797875000000001,-45.64827777777778,296.5034970667741,-71.3330336577803,[3],10.727745046324552,976.6579716976725,12m,,151.2,2013-04-25,"[95.42..97.30GHz,976.66kHz,7.3mJy/beam@10km/s,304.2uJy/beam@native, XX YY] U [97.30..99.17GHz,976.66kHz,7.2mJy/beam@10km/s,303uJy/beam@native, XX YY] U [107.42..109.30GHz,976.66kHz,7.5mJy/beam@10km/s,327.9uJy/beam@native, XX YY] U [109.30..111.17GHz,976.66kHz,7.8mJy/beam@10km/s,343.8uJy/beam@native, XX YY]",2633.9398359361962,XX YY,2011-11-19 04:46:11,"Weiss, Axel",Tuning4_1-2HA,"Chapman, Scott; Brodwin, Mark; Greve, Thomas; Hezaveh, Yashar; Sharon, Keren; Ashby, Matthew; Menten, Karl; Carlstrom, John; Marrone, Dan; De Breuck, Carlos; Rosenman, Michael; Holzapfel, William; Vieira, Joaquin; Crawford, Thomas; Lupu, Roxana; Fassnacht, Chris; Holder, Gilbert; Aguirre, James;",7.468943547865293,0.15921203104645965,4.161038,,uid://A001/X74/X293,uid://A002/X321719/X826,The ALMA-SPT Redshift Survey,S,TARGET WVR,60.988833626470466,43.958459712754184,Y,14,"Sub-mm Galaxies (SMG), Gravitational lenses",Galaxy evolution,2011.0.00957.S,SPT0103-45
2011.0.00957.S,SPT_0113-46,18.285375,-46.2963611111111,291.725192459293,-70.35680731865747,[3],7.806784683409882,976.657768509833,12m,,151.2,2013-04-25,"[91.67..93.55GHz,976.66kHz,7.4mJy/beam@10km/s,300.6uJy/beam@native, XX YY] U [93.55..95.42GHz,976.66kHz,7.3mJy/beam@10km/s,301.3uJy/beam@native, XX YY] U [103.67..105.55GHz,976.66kHz,7.3mJy/beam@10km/s,313.5uJy/beam@native, XX YY] U [105.55..107.42GHz,976.66kHz,7.3mJy/beam@10km/s,319.2uJy/beam@native, XX YY]",2725.895948314194,XX YY,2011-11-19 03:43:05,"Weiss, Axel",Tuning3_1-2HA,"Chapman, Scott; Brodwin, Mark; Greve, Thomas; Hezaveh, Yashar; Sharon, Keren; Ashby, Matthew; Menten, Karl; Carlstrom, John; Marrone, Dan; De Breuck, Carlos; Rosenman, Michael; Holzapfel, William; Vieira, Joaquin; Crawford, Thomas; Lupu, Roxana; Fassnacht, Chris; Holder, Gilbert; Aguirre, James;",7.266922806270144,0.15418158889140254,4.0281453,,uid:

In [11]:
uniqueALMAnames = np.unique(testQuery['ALMA sanitized source name'])

In [ ]:
from astropy.table import join, setdiff, unique, vstack
from astroquery.ned import Ned
# query NED for object redshifts
nedResult = list()
for sourceName in uniqueALMAnames:
    try:
        nedSearch = Ned.query_object(sourceName)
        nedSearch['ALMA sanitized source name'] = sourceName
        # doing this prevents vstack warnings
        nedSearch.meta = None
        nedResult.append(nedSearch)
    except Exception:
        pass
nedResult = vstack(nedResult)

In [ ]:
nedResult

In [ ]:
# store rows without matching name in NED
queryResultsNoNED = setdiff(testQuery, nedResult,
                                         keys='ALMA sanitized source name')

In [ ]:
# remove rows without redshifts in NED
blankZinds = nedResult['Redshift'].mask.nonzero()
blankZnames = nedResult['ALMA sanitized source name'][blankZinds]
nedResult.remove_rows(blankZinds)

In [ ]:
# store rows with matching name in NED but no z
# (this seems like a dumb approach)
blankZinds = list()
for i,row in enumerate(testQuery):
    if row['ALMA sanitized source name'] in blankZnames:
        blankZinds.append(i)
queryResultsNoNEDz = testQuery[blankZinds]

In [ ]:
# rectify this naming difference between NED and ALMA
nedResult.rename_column('DEC', 'Dec')


In [ ]:
# keep redshifts, positions too if we wanna check later
nedResult.keep_columns(['Object Name', 'RA', 'Dec', 'Redshift',
                        'ALMA sanitized source name'])



In [ ]:
from astropy import units as u
import string

def parseFrequencyRanges(queryResults):
    """parse frequency ranges for payload query"""
    targetFreqRanges = list()
    freqUnit = queryResults['Frequency support'].unit
    for i in range(len(queryResults)):
        freqStr = queryResults['Frequency support'][i]
        freqStr = freqStr.split('U')
        rowFreqRanges = list()
        for j in range(len(freqStr)):
            freqRange = freqStr[j].split(',')
            freqRange = freqRange[0].strip(' [')
            freqRange = freqRange.split('..')
            freqRange[1] = freqRange[1].strip(string.ascii_letters)
            freqRange = np.array(freqRange, dtype='float')
            rowFreqRanges.append(freqRange)
        targetFreqRanges.append(rowFreqRanges)
    queryResults['Frequency ranges'] = targetFreqRanges
    queryResults['Frequency ranges'].unit = freqUnit
    

In [ ]:
# parse frequencies
parseFrequencyRanges(testQuery)

testQuery

In [ ]:
# join NED redshift table and ALMA archive table based on name
ALMAnedResults = join(testQuery, nedResult,
                      keys='ALMA sanitized source name')



In [ ]:
# tidy up column names
ALMAnedResults.rename_column('Source name', 'ALMA source name')
ALMAnedResults.rename_column('RA_1', 'ALMA RA')
ALMAnedResults.rename_column('Dec_1', 'ALMA Dec')
ALMAnedResults.rename_column('Object Name', 'NED source name')
ALMAnedResults.rename_column('RA_2', 'NED RA')
ALMAnedResults.rename_column('Dec_2', 'NED Dec')
ALMAnedResults.rename_column('Redshift', 'NED Redshift')

In [ ]:
def _lineObserved(target_frequency, observed_frequency_ranges, linename=""):
        """Loop through the observed spectral windows to check if 
            target_frequency is covered by spectral setup"""
        
        # Initialize boolean line observed flag array (i.e., True = line frequency in archive spw coverage)
        lineObserved = []
        
        # loop over spectral window frequencies for each observation
        for spw in observed_frequency_ranges:
            # if observed frequency is in spw, flag as True and break inner loop (move on to next observation) 
            if spw[0] <= target_frequency <= spw[-1]:
                print(linename,"observed frequency", target_frequency, "GHz",
                          "in range", spw[0], "-", spw[-1])
                lineObserved.append(True) # line IS observed
            else:
                lineObserved.append(False) # line NOT observed
            
        # Boolean line observed flag for each observation 
        if True in lineObserved:
            return True
        else:
            return False


In [ ]:
# mark flags if spw is on line (initialized to False)
lineObserved = np.zeros((len(ALMAnedResults), len(restFreqs)),
                         dtype=bool)

for i, row in enumerate(ALMAnedResults):

    # target redshift
    z = row['NED Redshift']

    # observed frequencies
    observed_frequencies = [_observedFreq(rf, row['NED Redshift']) for rf in restFreqs]

    if line_names == "":
        line_names = ['Line{:}'.format(i) for i in range(len(restFreqs))]

    # loop over the target lines, return a boolean flag array and add it to astropy table
    for j, (observed_frequency, linename) in enumerate(zip(observed_frequencies,line_names)):
        lineObserved[i, j]=_lineObserved(target_frequency=observed_frequency
                                                    , observed_frequency_ranges=row['Frequency ranges']
                                                    , linename=linename)

In [ ]:
# add flag columns to array
for i in range(len(restFreqs)):
                    ALMAnedResults[line_names[i]] = lineObserved[:, i]


In [ ]:
# remove rows which have no lines covered
lineCount = np.array(ALMAnedResults[line_names[0]], dtype=int)
for i in range(1, len(restFreqs)):
    lineCount += np.array(ALMAnedResults[line_names[i]], dtype=int)
noLinesInds = np.where(lineCount == 0)
ALMAnedResults.remove_rows(noLinesInds)

In [ ]:
queryResults = ALMAnedResults

In [ ]:
queryResults

In [ ]:
print("galaxies with 12CO, 13CO, OR C18O:", len(queryResults))

In [ ]:
idx = np.where((True in queryResults['12CO J=1-0']) & 
    (True in queryResults['13CO J=1-0']) &
    (True in queryResults['C18O J=1-0']))

display(queryResults[idx])


In [ ]:
print("galaxies with 12CO, 13CO, AND C18O:", k)

Reload `ALMAxmatch` using IPythons `%autoreload` extenstion

In [ ]:
%load_ext autoreload
%autoreload

In [ ]:
from ALMAxmatch import archiveSearch

### runPayloadQuery Test

In [ ]:
payload = {'frequency':freqLimits, }

payload

In [ ]:
testPayloadQuery = archiveSearch()

In [ ]:
testPayloadQuery.runPayloadQuery(payload=payload,science=True)

In [ ]:
testPayloadQuery.queryResults

## runPayloadQueryWithLines test

In [ ]:
testPayloadLinesQuery = archiveSearch()

In [ ]:
testPayloadLinesQuery.runPayloadQueryWithLines(restFreqs=rest_frequencies,
                                          redshiftRange=redshift_range,
                                          line_names=line_names, science=True)

In [ ]:
testPayloadLinesQuery.queryResults

In [ ]:
print(len(testPayloadLinesQuery.queryResults), "observations of 12CO, 13CO, OR C18O:")